In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from models import FineTuneBaseModel
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 6, 5, 3, 8, 2),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [13]:
print(db.query("SELECT POST_ID FROM POSTS WHERE TARGET IN (0, 1)"))

[('nnpn9z',), ('nns6mk',), ('nn4aej',), ('nmpk6k',), ('nmjo2l',), ('nmabst',), ('nm53o9',), ('nmthag',), ('nmpq6d',)]


In [13]:
print(db.query('''SELECT * FROM STOCKS ;'''))

[('GME', datetime.datetime(2021, 5, 31, 18, 8, 14))]


In [3]:
db.query('''SELECT TITLE, TARGET FROM POSTS;''')

[("Current Price Action of GME is shockingly similar to the past two major run-ups and 'micro squeezes'",
  1),
 ("Added $165K more to the GME July 800s YOLO - $646K at risk let's GOOOOO",
  0),
 ('Ape wonders about AMC,GME and BB advertising budget 🙉🚀 SpaceX', 1),
 ('Another $GME, $AMC, $BB aimed for the MOON', 0),
 ('We like the stock $AMC $GME', 1),
 ('Is $SNDL going to do the amc/gme? 🚀', 0),
 ('GME to the moon!!! And sometimes bananas fuck assholes!!! 💎🙌🦍', 1),
 ('Lets Go!!! GME to the moon!!!', 0),
 ('The Gang Buys $GME', 1),
 ('Fresh out of the womb tube here gentlemen! I missed GME. I missed AMC. Putting my toes in the water',
  0)]

In [4]:
db.query('''SELECT COMMENT, TARGET FROM COMMENTS;''')

[("6/9....it's only fitting", 0),
 ('Damn I love GME', 0),
 ("Somethin ain't right here...  \n\n\n  Your wife has a *girlfriend*?", 1),
 ("This guy has a fully charged phone?!?!?! Something doesn't add up", 0),
 ('BB is buckling up. Let’s go 🚀🚀🚀', 1),
 ('Lol, this would be great.\nTweet @KatStryker111 she is the ape advertisement queen',
  0),
 ('Busy, check back in 2 weeks.', 1),
 ('"TO RISE FOR THE NEXT DECADE"  \n\n\nBitch if it aint rising by next week then I DONT WANT IT.',
  0),
 ('Here before this gets yeeted', 1),
 ('280$ next week just like that guy predicted', 0),
 ('We want $SNDL  \nWe want $SNDL  \nWe want $SNDL', 1),
 ("I don't know if I can trust you with a fully charged battery. 🤔", 0),
 ('Name me a better motivational speech... I dare you....', 1),
 ('Now I have to watch team America again, Trey and Matt are fucking legendary!!!!!',
  0),
 ('FUCK YEAH!!!', 1),
 ('Stonk Market Ape Police', 0),
 ('I’ve seen so many memes of this show that I may just as well start watching

In [39]:
db.query("SELECT TITLE FROM POSTS WHERE POST_ID='{}' AND STOCK_ID='{}'".format(indexes[0][0], 'GME'))[0][0]

"I, too, YOLO'D my retirement account into GME when I switched jobs. PICK ME UP ON THE WAY TO THE MOO"

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [62]:
db.query("SELECT COUNT(*) FROM POSTS WHERE StOCK_ID='{}';".format("GME"))[0][0]

10

In [18]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 10, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 6, 5, 0, 40, 6),)]
connection established
[(datetime.datetime(2021, 6, 5, 0, 40, 7),)]


  0%|          | 0/10 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 5, 0, 40, 9),)]


100%|██████████| 10/10 [01:45<00:00, 10.50s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [8]:
print(db.query('show tables;'))

[('POSTS',)]


In [5]:
print(db.query('''SELECT POST_ID FROM POSTS WHERE TARGET=-1 ;'''))

[('nnpn9z',), ('nn4aej',), ('nns6mk',), ('nmpk6k',), ('nmjo2l',), ('nm53o9',), ('nmabst',), ('nmthag',), ('nmmu24',), ('nmpq6d',)]


In [4]:
print(db.query('''SHOW COLUMNS FROM LABELED_POSTS;'''))

[('POST_ID', b'char(20)', 'NO', '', None, ''), ('TARGET', b'int', 'YES', '', None, ''), ('TITLE', b'char(100)', 'YES', '', None, '')]


In [4]:
db.drop_table("LABELED_POSTS")
db.drop_table("LABELED_COMMENTS")

In [15]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [2]:
indices = get_indices("no_stock_id")

connection established
[(datetime.datetime(2021, 6, 2, 17, 22, 23),)]


In [3]:
post_train = PostDataset(512, indices['post_train'])



connection established
[(datetime.datetime(2021, 6, 2, 17, 22, 24),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/ianleefmans/opt/anaconda3/envs/StockSentiment/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [6]:
sample


{'post': ['GME & AMC',
  'What if GME does a OSTK.',
  'Proper 401k weighting for a 26 year old? 99.8% $GME, 0.2% $AMC'],
 'post_input_ids': tensor([[  101, 14748,  2036,  ...,     0,     0,     0],
         [  101,  1327,  1191,  ...,     0,     0,     0],
         [  101,  5096,  3365,  ...,     0,     0,     0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([[[0., 1.]],
 
         [[1., 0.]],
 
         [[0., 1.]]])}

In [16]:
model = FineTuneBaseModel()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'torch.Tensor'>
tensor([[-0.6015,  0.3814,  0.9997,  ...,  0.9999, -0.6898,  0.9579],
        [-0.6080,  0.4887,  0.9999,  ...,  1.0000, -0.9328,  0.9946],
        [-0.5415,  0.3671,  0.9996,  ...,  0.9998, -0.7365,  0.9712]],
       grad_fn=<TanhBackward>)


In [17]:
out

tensor([[0.7030, 0.2970],
        [0.7028, 0.2972],
        [0.7087, 0.2913]], grad_fn=<SoftmaxBackward>)